In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
if 'COLAB_GPU' in os.environ:
    from google.colab import  drive
    drive.mount('/drive')
    data_path = '/drive/Shared drives/Capstone/notebooks/data'
else:
    data_path = 'data'

### Presidential election 2020

sources

- https://github.com/MEDSL
- https://github.com/MEDSL/2020-elections-official

In [3]:
# source: 
#  https://github.com/MEDSL/2020-elections-official/blob/main/all_states/2020-all-states-local-precinct-general.zip

In [4]:
# https://dataverse.harvard.edu/file.xhtml?fileId=6100388&version=1.1 
# readme and notes: https://github.com/MEDSL/2020-elections-official
# can't download via api 
pres_df =  pd.read_csv(f'{data_path}/raw/2020-PRESIDENT-precinct-general.zip', compression='zip')

In [5]:
pres_df.head(10)

,precinct,office,party_detailed,party_simplified,mode,votes,county_name,county_fips,jurisdiction_name,jurisdiction_fips,...,stage,state,special,writein,state_po,state_fips,state_cen,state_ic,date,readme_check
0,10 JONES COMM_ CTR_,US PRESIDENT,NaN,NaN,TOTAL,2,AUTAUGA,1001.0,AUTAUGA,1001.0,...,GEN,ALABAMA,False,False,AL,1,63,41,2020-11-03,False
1,10 JONES COMM_ CTR_,US PRESIDENT,NaN,NaN,TOTAL,0,AUTAUGA,1001.0,AUTAUGA,1001.0,...,GEN,ALABAMA,False,False,AL,1,63,41,2020-11-03,False
2,10 JONES COMM_ CTR_,US PRESIDENT,NaN,NaN,TOTAL,0,AUTAUGA,1001.0,AUTAUGA,1001.0,...,GEN,ALABAMA,False,True,AL,1,63,41,2020-11-03,False
3,10 JONES COMM_ CTR_,US PRESIDENT,DEMOCRAT,DEMOCRAT,TOTAL,141,AUTAUGA,1001.0,AUTAUGA,1001.0,...,GEN,ALABAMA,False,False,AL,1,63,41,2020-11-03,False
4,10 JONES COMM_ CTR_,US PRESIDENT,INDEPENDENT,OTHER,TOTAL,3,AUTAUGA,1001.0,AUTAUGA,1001.0,...,GEN,ALABAMA,False,False,AL,1,63,41,2020-11-03,False
5,10 JONES COMM_ CTR_,US PRESIDENT,REPUBLICAN,REPUBLICAN,TOTAL,226,AUTAUGA,1001.0,AUTAUGA,1001.0,...,GEN,ALABAMA,False,False,AL,1,63,41,2020-11-03,False
6,100 TRINITY METHODIST,US PRESIDENT,NaN,NaN,TOTAL,1,AUTAUGA,1001.0,AUTAUGA,1001.0,...,GEN,ALABAMA,False,False,AL,1,63,41,2020-11-03,False
7,100 TRINITY METHODIST,US PRESIDENT,NaN,NaN,TOTAL,4,AUTAUGA,1001.0,AUTAUGA,1001.0,...,GEN,ALABAMA,False,False,AL,1,63,41,2020-11-03,False
8,100 TRINITY METHODIST,US PRESIDENT,NaN,NaN,TOTAL,7,AUTAUGA,1001.0,AUTAUGA,1001.0,...,GEN,ALABAMA,False,True,AL,1,63,41,2020-11-03,False
9,100 TRINITY METHODIST,US PRESIDENT,DEMOCRAT,DEMOCRAT,TOTAL,315,AUTAUGA,1001.0,AUTAUGA,1001.0,...,GEN,ALABAMA,False,False,AL,1,63,41,2020-11-03,False


In [6]:
# verifying we have no indiana data , that section is blank on the map. 
# pres_df[pres_df['state']=='INDIANA']

In [7]:
#drop nans for fips 41055 and 36017 due to duplicate rows, NAN in party_simplified, and party_detailed. 
# The NAN add up to a total of the other rows 
# in the same precinct and fips with a value in party_detailed and party_detailed 
# so we remove the nan rows to remove duplicates
pres_df[pres_df['county_fips']==41055] = pres_df[pres_df['county_fips']==41055].dropna(subset=['party_simplified'])
pres_df[pres_df['county_fips']==36017.0] = pres_df[pres_df['county_fips']==36017.0].dropna(subset=['party_simplified'])


In [8]:
# one way to check if the nan total column is a total of the other columns is to check the summed values 
# create a function  

nans_win_df = pres_df[pres_df['party_simplified'].isnull()].groupby(['precinct', 'county_fips'])['votes'].sum().reset_index()
total_win_df =  pres_df[~pres_df['party_simplified'].isnull()].groupby(['precinct', 'county_fips'])['votes'].sum().reset_index()

In [9]:
nans_win_df

,precinct,county_fips,votes
0,2021310007,8019.0,0.0
1,2021310008,8019.0,0.0
2,2021310009,8019.0,0.0
3,2026047008,8093.0,0.0
4,2026047010,8093.0,0.0
...,...,...,...
70487,ZUMBRO FALLS,27157.0,0.0
70488,ZUMBRO TWP.,27157.0,0.0
70489,ZUMBROTA P-1,27049.0,7.0
70490,ZUMBROTA P-2,27049.0,2.0


In [10]:
merged_df = total_win_df.merge(nans_win_df, how='left', left_on=['precinct', 'county_fips'], right_on=['precinct', 'county_fips'])

In [11]:
# check if the NANs votes_y are substancially higher or the same as the votes_x
# if these numbers are the same or votes_y is higher check more data. Nothing extremely out of the ordinary
# precinct 99999 is for statistically changes/updates by fips code
merged_df[(merged_df['votes_x']<merged_df['votes_y'])&(merged_df['votes_x']>0)]

,precinct,county_fips,votes_x,votes_y
105502,9999,26065.0,9.0,45.0
119687,CUMULATIVE,36123.0,7.0,8.0


In [12]:
pres_df[(pres_df['precinct']=='9999')&(pres_df['county_fips']==26065.0)]

,precinct,office,party_detailed,party_simplified,mode,votes,county_name,county_fips,jurisdiction_name,jurisdiction_fips,...,stage,state,special,writein,state_po,state_fips,state_cen,state_ic,date,readme_check
1107304,9999,US PRESIDENT,NaN,NaN,TOTAL,43.0,INGHAM,26065.0,{STATISTICAL ADJUSTMENTS},26065.0,...,GEN,MICHIGAN,False,True,MI,26.0,34.0,23.0,2020-11-03,True
1107335,9999,US PRESIDENT,NaN,NaN,TOTAL,2.0,INGHAM,26065.0,{STATISTICAL ADJUSTMENTS},26065.0,...,GEN,MICHIGAN,False,True,MI,26.0,34.0,23.0,2020-11-03,True
1107355,9999,US PRESIDENT,NaN,NaN,TOTAL,0.0,INGHAM,26065.0,{STATISTICAL ADJUSTMENTS},26065.0,...,GEN,MICHIGAN,False,True,MI,26.0,34.0,23.0,2020-11-03,True
1107389,9999,US PRESIDENT,NaN,NaN,TOTAL,0.0,INGHAM,26065.0,{STATISTICAL ADJUSTMENTS},26065.0,...,GEN,MICHIGAN,False,True,MI,26.0,34.0,23.0,2020-11-03,True
1107420,9999,US PRESIDENT,NaN,NaN,TOTAL,0.0,INGHAM,26065.0,{STATISTICAL ADJUSTMENTS},26065.0,...,GEN,MICHIGAN,False,True,MI,26.0,34.0,23.0,2020-11-03,True
1107473,9999,US PRESIDENT,DEMOCRAT,DEMOCRAT,TOTAL,3.0,INGHAM,26065.0,{STATISTICAL ADJUSTMENTS},26065.0,...,GEN,MICHIGAN,False,False,MI,26.0,34.0,23.0,2020-11-03,True
1107491,9999,US PRESIDENT,LIBERTARIAN,LIBERTARIAN,TOTAL,0.0,INGHAM,26065.0,{STATISTICAL ADJUSTMENTS},26065.0,...,GEN,MICHIGAN,False,False,MI,26.0,34.0,23.0,2020-11-03,True
1107525,9999,US PRESIDENT,US TAXPAYERS PARTY,OTHER,TOTAL,0.0,INGHAM,26065.0,{STATISTICAL ADJUSTMENTS},26065.0,...,GEN,MICHIGAN,False,False,MI,26.0,34.0,23.0,2020-11-03,True
1107559,9999,US PRESIDENT,GREEN,OTHER,TOTAL,0.0,INGHAM,26065.0,{STATISTICAL ADJUSTMENTS},26065.0,...,GEN,MICHIGAN,False,False,MI,26.0,34.0,23.0,2020-11-03,True
1107612,9999,US PRESIDENT,NATURAL LAW PARTY,OTHER,TOTAL,1.0,INGHAM,26065.0,{STATISTICAL ADJUSTMENTS},26065.0,...,GEN,MICHIGAN,False,False,MI,26.0,34.0,23.0,2020-11-03,True


In [13]:
pres_df[(pres_df['party_simplified'].isnull())&(pres_df['votes']==7937)]

# example of some rows that have more votes than population 
# fips: 49009, votes: 619 , total voter pop: 440.0, voter proprotion: 	1.406818

# fips:	35021, voter pop: 347.0, votes: 483.0, voter proportion:	1.391931

#fips: 8111, voting pop: 500.0, votes:		562.0, voter proportion:	1.124

# fips:	48311, voting pop: 475.0, votes:	516.0, voter proportion:	1.086316

# fips: 8079, voter pop: 724.0,, votes:		756.0, voter proportion:	1.044199

,precinct,office,party_detailed,party_simplified,mode,votes,county_name,county_fips,jurisdiction_name,jurisdiction_fips,...,stage,state,special,writein,state_po,state_fips,state_cen,state_ic,date,readme_check
1851627,COUNTYWIDE,US PRESIDENT,NaN,NaN,TOTAL,7937.0,KING,53033.0,KING,53033.0,...,GEN,WASHINGTON,False,True,WA,53.0,91.0,73.0,2020-11-03,False


In [14]:
# basic sum of county fips votes,
# some jurisdictions include TOTAL but others break down by absentee, inperson etc. It seems to be one way or the other

sum_votes_df = pres_df.groupby(['county_fips'])['votes'].sum().reset_index().rename(columns={'county_fips': 'FIPS', 'votes': 'TOTAL_VOTES'})

In [15]:
voting_df = pd.read_csv(f'{data_path}/interim/voting_population_2020_acs.csv' )
#voting_df['FIPS_state'] = voting_df['state'].astype(str).str.zfill(2)
voting_df['FIPS_county'] = voting_df['county'].astype(str).str.zfill(3)
voting_df['FIPS'] = (voting_df['state'].astype(str) + voting_df['FIPS_county']).astype(int)
voting_df.drop(['FIPS_county','state', 'county'], axis=1, inplace=True)
voting_df.head()


,voting_population,FIPS
0,41959.0,1001
1,167986.0,1003
2,19468.0,1005
3,17583.0,1007
4,42978.0,1009


In [16]:
print(len(voting_df))

3221


In [17]:
voter_percent_df = voting_df.merge(sum_votes_df, how='left', left_on='FIPS', right_on='FIPS')

In [18]:
voter_percent_df.head()
voter_percent_df['proportion_voter'] = voter_percent_df['TOTAL_VOTES']/voter_percent_df['voting_population'] 

In [19]:
voter_percent_df[voter_percent_df['proportion_voter'].isnull()]

,voting_population,FIPS,TOTAL_VOTES,proportion_voter
619,267051.0,18003,NaN,NaN
620,9242.0,18009,NaN,NaN
621,15442.0,18015,NaN,NaN
622,20049.0,18021,NaN,NaN
623,22901.0,18027,NaN,NaN
...,...,...,...,...
2271,29443.0,72131,NaN,NaN
2272,60621.0,72137,NaN,NaN
2273,28758.0,72143,NaN,NaN
2274,26747.0,72151,NaN,NaN


In [20]:
# number of counties with proportion of voters greater than 1
len(voter_percent_df[voter_percent_df['proportion_voter']> 1])/len(voter_percent_df)

0.0018627755355479665

In [21]:
# Drop any vote proportion higher than 1 
voter_percent_df = voter_percent_df[voter_percent_df['proportion_voter']<= 1]



In [22]:
print(len(voter_percent_df))

3013


In [23]:
voter_percent_df.head()
export_voter_df = voter_percent_df[['FIPS', 'proportion_voter']]
export_voter_df.describe()

,FIPS,proportion_voter
count,3013.000000,3013.000000
mean,31026.526054,0.657604
std,15041.391817,0.099756
min,1001.000000,0.194230
25%,19177.000000,0.589751
50%,30065.000000,0.659572
75%,46047.000000,0.727149
max,56045.000000,0.998771


In [24]:
export_voter_df[['FIPS', 'proportion_voter']].sort_values(by='proportion_voter', ascending=False).head(10)

,FIPS,proportion_voter
881,30033,0.998771
857,8091,0.974073
841,8053,0.951807
644,25007,0.949757
832,8035,0.947159
2280,48033,0.945455
2307,26089,0.943344
2997,38007,0.940741
776,53055,0.938086
834,8039,0.933743


In [25]:
print(len(export_voter_df['FIPS'].unique()))

3013


In [26]:
export_voter_df.to_csv(f'{data_path}/processed/presidential_precinct_general.csv', index=False)

In [27]:
# just to verify 
totalvotes = sum_votes_df['TOTAL_VOTES'].sum()
# Americans voted in 2020: 159,633,396 
# https://www.cfr.org/blog/2020-election-numbers
# biden 81,283,098 + trump 74,222,958 = 155506056 
# The numbers seem reasonable 

In [28]:
totalvotes

155234081.0